In [26]:
#version 4 changes the smoothing rule
#version 6 accounts for growth of the economy 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [27]:
#simulation

#generate random errors
sample_size = 1000000
T = 80 #years
std_norm_e = np.random.randn(T*sample_size, 3) #np.random.multivariate_normal([0,0,0], cov_e, (T, sample_size))

In [28]:
#load VAR parameter values from excel sheet
VAR_param = pd.read_excel('VAR_param.xlsx', index_col=0)
VAR_param

,Real NOK 70-30,Real NOK 50-50,Real USD 70-30,Real USD 50-50
r_0,0.063200,0.058200,0.049100,0.044100
a_1,-0.037500,-0.037300,-0.059900,-0.075600
a_2,1.192300,1.192310,1.199200,1.204300
a_3,-0.506900,-0.515700,-0.489200,-0.496900
b_1,1.082700,1.084700,1.073100,1.069400
b_2,-0.295000,-0.297800,-0.283500,-0.283700
std_r,0.138900,0.116800,0.123900,0.093400
std_x,0.017100,0.017600,0.015700,0.016000
std_f,0.021800,0.021800,0.021900,0.021900
cov_xf,0.000148,0.000155,0.000119,0.000132


In [29]:
#prepare parameter values
r_0, a_1, a_2, a_3, b_1, b_2, std_r, std_x, std_f, cov_xf = VAR_param['Real NOK 70-30'][:-1]
b_0 = 0


r_mean = 0.03
mu = np.log(1+r_mean)

corr_rx = corr_rf = 0
#cov_xf = 0.7*std_x*std_f

cov_e = [[std_r**2, corr_rx*std_r*std_x, corr_rf*std_r*std_f],
        [corr_rx*std_r*std_x, std_x**2, cov_xf],
        [corr_rf*std_r*std_f, cov_xf, std_f**2]]
cholesky_cov = np.linalg.cholesky(cov_e)


In [30]:
#simulate the VAR
#transform errors
growth_factor = 1.014
e=cholesky_cov.dot(std_norm_e.T).T.reshape(T, sample_size, 3)

r = np.zeros((T, sample_size))
X = np.zeros((T, sample_size))
F = np.zeros((T, sample_size))

#first simulate r because it is independent
r = mu -.5*std_r**2 + e[:,:,0]
#then simulate X and F
#note that the first couple of values are taken from end of arrays
for t in range(T):
    X[t] = a_1*r[t-1] + a_2*X[t-1] + a_3*X[t-2] + e[t,:,1]*growth_factor**t
    F[t] = b_0*r[t-1] + b_1*F[t-1] + b_2*F[t-2] + e[t,:,2]*growth_factor**t

In [31]:
#simulation function
def sim(A_0, X, F, params, new_smoothing):
    df = pd.DataFrame(index = ['5%', '25%', '50%', '75%', '95%', 'Mean','Depletion rate']) #table to save results to

    for param in params:
        lambda_ = param['lambda']
        A = np.zeros((T, sample_size))
        D = np.zeros((T, sample_size))
        S = np.zeros((T, sample_size))
        A[0] = A_0
        S[0] = r_mean*A[0]
        D[0] = S[0] + X[0]*param['X'] + F[0]*param['F']
        for t in range(1,T):
            A[t] = np.maximum(A[t-1]*np.exp(r[t-1])-D[t-1], 0)

            if new_smoothing:
                S[t] = np.where(S[t-1]<=r_mean*A[t], r_mean*A[t]+F[t]*param['F'], lambda_*S[t-1]+(1-lambda_)*r_mean*A[t]+F[t]*param['F'])
                #S[t] = np.maximum(r_mean*A[t]+F[t]*param['F'], lambda_*S[t-1]+(1-lambda_)*r_mean*A[t]+F[t]*param['F'])
            else:    
                S[t] = np.where(S[t-1]<=r_mean*A[t-1], r_mean*A[t]+F[t]*param['F'], lambda_*S[t-1]+(1-lambda_)*r_mean*A[t]+F[t]*param['F']) 
            D[t] = S[t] + X[t]*param['X']

        df[r'$\lambda$={}, X={}, F={}'.format(param['lambda'], param['X'], param['F'])] = [np.percentile(A[-1], 5),np.percentile(A[-1], 25), np.percentile(A[-1], 50), np.percentile(A[-1], 75), np.percentile(A[-1], 95), A[-1].mean(),(A[-1]==0).mean()]
    return df



In [32]:
#NOK 70-30
params = [{'lambda':0, 'F':False, 'X':False},
          {'lambda':0, 'F':False, 'X':True},
          {'lambda':0, 'F':True, 'X':True},
         {'lambda':0.5, 'F':True, 'X':True},
         {'lambda':0.8, 'F':True, 'X':True}]

sim(12, X, F, params, new_smoothing = True)

,"$\lambda$=0, X=False, F=False","$\lambda$=0, X=True, F=False","$\lambda$=0, X=True, F=True","$\lambda$=0.5, X=True, F=True","$\lambda$=0.8, X=True, F=True"
5%,0.659182,0.475944,0.141875,0.000000,0.000000
25%,2.262683,2.277603,2.242308,0.993669,0.000000
50%,5.340629,5.460578,5.593339,3.764701,1.099589
75%,12.611073,12.874634,13.085214,10.327050,5.217008
95%,43.333241,44.122237,44.392995,38.805946,28.672601
Mean,11.990239,12.186535,12.262627,9.976552,6.398478
Depletion rate,0.000000,0.010681,0.034500,0.101459,0.279548


In [33]:
#greater a_1 and b_0
a_1 = -0.09
b_0 = a_1


r = np.zeros((T, sample_size))
X = np.zeros((T, sample_size))
F = np.zeros((T, sample_size))

#first simulate r because it is independent
r = mu -.5*std_r**2 + e[:,:,0]
#then simulate X and F
#note that the first couple of values are taken from end of arrays
for t in range(T):
    X[t] = a_1*r[t-1] + a_2*X[t-1] + a_3*X[t-2] + e[t,:,1]*growth_factor**t
    F[t] = b_0*r[t-1] + b_1*F[t-1] + b_2*F[t-2] + e[t,:,2]*growth_factor**t

    
params = [{'X': True, 'F': True, 'lambda': 0},
         {'X': True, 'F': True, 'lambda': 0.5},
         {'X': True, 'F': True, 'lambda': 0.8}]

sim(12, X, F, params, new_smoothing = True)

,"$\lambda$=0, X=True, F=True","$\lambda$=0.5, X=True, F=True","$\lambda$=0.8, X=True, F=True"
5%,0.135117,0.000000,0.000000
25%,2.318783,1.123340,0.015753
50%,5.994128,4.201632,1.515757
75%,14.179468,11.582237,6.696396
95%,47.889529,42.909928,33.915511
Mean,13.187972,11.050891,7.671000
Depletion rate,0.034873,0.093880,0.245064


In [34]:
#greater a_1 and b_0 and stronger correlation between the markets and norwegian economy
a_1 = -0.09
b_0 = a_1


corr_rx = corr_rf = -0.5
cov_xf = 0.7*std_x*std_f

cov_e = [[std_r**2, corr_rx*std_r*std_x, corr_rf*std_r*std_f],
        [corr_rx*std_r*std_x, std_x**2, cov_xf],
        [corr_rf*std_r*std_f, cov_xf, std_f**2]]
cholesky_cov = np.linalg.cholesky(cov_e)
e=cholesky_cov.dot(std_norm_e.T).T.reshape(T, sample_size, 3)

r = np.zeros((T, sample_size))
X = np.zeros((T, sample_size))
F = np.zeros((T, sample_size))

#first simulate r because it is independent
r = mu -.5*std_r**2 + e[:,:,0]
#then simulate X and F
#note that the first couple of values are taken from end of arrays
for t in range(T):
    X[t] = a_1*r[t-1] + a_2*X[t-1] + a_3*X[t-2] + e[t,:,1]*growth_factor**t
    F[t] = b_0*r[t-1] + b_1*F[t-1] + b_2*F[t-2] + e[t,:,2]*growth_factor**t

    
params = [{'X': True, 'F': True, 'lambda': 0},
         {'X': True, 'F': True, 'lambda': 0.5},
         {'X': True, 'F': True, 'lambda': 0.8}]

sim(12, X, F, params, new_smoothing = True)

,"$\lambda$=0, X=True, F=True","$\lambda$=0.5, X=True, F=True","$\lambda$=0.8, X=True, F=True"
5%,0.000000,0.000000,0.000000
25%,1.716535,0.695452,0.000000
50%,5.558544,3.568909,1.231605
75%,14.377244,11.340644,6.032369
95%,50.350173,44.871900,34.988663
Mean,13.352371,11.062226,7.641270
Depletion rate,0.062622,0.131233,0.282964


In [35]:
# old smoothing rule, greater a_1 and b_0 and stronger correlation between the markets and norwegian economy
sim(12, X, F, params, new_smoothing = False)

,"$\lambda$=0, X=True, F=True","$\lambda$=0.5, X=True, F=True","$\lambda$=0.8, X=True, F=True"
5%,0.000000,0.000000,0.000000
25%,1.716535,0.713661,0.000000
50%,5.558544,3.651054,1.304739
75%,14.377244,11.637000,6.513061
95%,50.350173,46.282466,37.944475
Mean,13.352371,11.400190,8.281916
Depletion rate,0.062622,0.129851,0.278175


In [36]:
#small fund new smoothing rule, greater a_1 and b_0 and stronger correlation between the markets and norwegian economy
sim(3, X, F, params, new_smoothing = True)

,"$\lambda$=0, X=True, F=True","$\lambda$=0.5, X=True, F=True","$\lambda$=0.8, X=True, F=True"
5%,0.000000,0.000000,0.000000
25%,0.528760,0.130514,0.000000
50%,2.120559,1.324144,0.553819
75%,5.486865,4.022880,2.450598
95%,18.568757,15.523985,11.192906
Mean,4.915892,3.851233,2.633065
Depletion rate,0.119274,0.203772,0.348059


In [37]:
#small fund old smoothing rule, greater a_1 and b_0 and stronger correlation between the markets and norwegian economy
sim(3, X, F, params, new_smoothing = False)

,"$\lambda$=0, X=True, F=True","$\lambda$=0.5, X=True, F=True","$\lambda$=0.8, X=True, F=True"
5%,0.000000,0.000000,0.000000
25%,0.528760,0.132154,0.000000
50%,2.120559,1.330983,0.561536
75%,5.486865,4.048564,2.479893
95%,18.568757,15.715832,11.502501
Mean,4.915892,3.894699,2.704807
Depletion rate,0.119274,0.203362,0.346956
